# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si, Si]
positions =  [ones(3)/8, -ones(3)/8]

# We take very (very) crude parameters
model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[1, 1, 1]);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, max_iter; tol)
    mixing_factor = .7
    x = x0
    fx = f(x)
    for n = 1:max_iter
        inc = fx - x
        if norm(inc) < tol
            break
        end
        x = x + mixing_factor * inc
        fx = f(x)
    end
    (fixpoint=x, converged=norm(fx-x) < tol)
end;

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (; λ, X, residual_norms=[], iterations=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.
In the mixing interface `δF` is $(ρ_\text{out} - ρ_\text{in})$, i.e.
the difference in density between two subsequent SCF steps and the `mix`
function returns $δρ$, which is added to $ρ_\text{in}$ to yield $ρ_\text{next}$,
the density for the next SCF step.

In [4]:
struct MyMixing
    n_simple  # Number of iterations for simple mixing
end
MyMixing() = MyMixing(2)

function DFTK.mix_density(mixing::MyMixing, basis, δF; n_iter, kwargs...)
    if n_iter <= mixing.n_simple
        return δF  # Simple mixing -> Do not modify update at all
    else
        # Use the default KerkerMixing from DFTK
        DFTK.mix_density(KerkerMixing(), basis, δF; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-8,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.216581075811                   -0.47    0.0         
  2   -7.246549399868       -1.52       -0.85    0.0    600ms
  3   -7.251006610304       -2.35       -1.29    0.0    181ms
  4   -7.251258709952       -3.60       -1.60    0.0   85.6ms
  5   -7.251318976989       -4.22       -1.90    0.0   84.6ms
  6   -7.251333669324       -4.83       -2.20    0.0   84.8ms
  7   -7.251337397659       -5.43       -2.48    0.0   84.4ms
  8   -7.251338393932       -6.00       -2.76    0.0    188ms
  9   -7.251338675504       -6.55       -3.04    0.0   84.1ms
 10   -7.251338759506       -7.08       -3.30    0.0   85.9ms
 11   -7.251338785787       -7.58       -3.56    0.0   86.4ms
 12   -7.251338794333       -8.07       -3.81    0.0   87.4ms
 13   -7.251338797196       -8.54       -4.05    0.0   86.2ms
 14   -7.251338798177       -9.01       -4.29    0.0    176ms
 15   -7.

Note that the default convergence criterion is the difference in
density. When this gets below `tol`, the
"driver" `self_consistent_field` artificially makes the fixed-point
solver think it's converged by forcing `f(x) = x`. You can customize
this with the `is_converged` keyword argument to
`self_consistent_field`.